# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [3]:
# Import the data
import pandas as pd
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [4]:
# Your code here
minority = df[df['label'] == 'spam']
undersampled_majority = df[df['label'] == 'ham'].sample(n=len(minority))
df2 = pd.concat([minority, undersampled_majority])
df2.label.value_counts()

ham     747
spam    747
Name: label, dtype: int64

In [5]:
p_classes = dict(df2['label'].value_counts(normalize=True))
p_classes

{'ham': 0.5, 'spam': 0.5}

In [6]:
df2.iloc[0]

label                                                 spam
text     Free entry in 2 a wkly comp to win FA Cup fina...
Name: 2, dtype: object

## Train-test split

Now implement a train-test split on the dataset: 

In [7]:
from sklearn.model_selection import train_test_split
X = df2['text']
y = df2['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
train_df = pd.concat([X_train, y_train], axis=1) 
test_df = pd.concat([X_test, y_test], axis=1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [8]:
# Will be a nested dictionary of class_i : {word1:freq, word2:freq..., wordn:freq},.... class_m : {}
class_word_freq = {} 
classes = train_df['label'].unique()
for class_ in classes:
    temp_df = train_df[train_df['label'] == class_]
    bag = {}
    for row in temp_df.index:
        doc = temp_df['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    class_word_freq[class_] = bag

In [11]:
class_word_freq['spam']


{'Think': 2,
 'ur': 73,
 'smart': 1,
 '?': 5,
 'Win': 9,
 '£200': 6,
 'this': 37,
 'week': 21,
 'in': 51,
 'our': 59,
 'weekly': 13,
 'quiz,': 1,
 'text': 49,
 'PLAY': 2,
 'to': 453,
 '85222': 1,
 'now!T&Cs': 1,
 'WinnersClub': 1,
 'PO': 17,
 'BOX': 7,
 '84,': 1,
 'M26': 3,
 '3UZ.': 1,
 '16+.': 3,
 'GBP1.50/week': 1,
 'Urgent!': 8,
 'call': 140,
 '09066350750': 1,
 'from': 86,
 'your': 147,
 'landline.': 13,
 'Your': 53,
 'complimentary': 11,
 '4*': 20,
 'Ibiza': 3,
 'Holiday': 18,
 'or': 142,
 '10,000': 1,
 'cash': 33,
 'await': 16,
 'collection': 9,
 'SAE': 13,
 'T&Cs': 16,
 '434': 2,
 'SK3': 2,
 '8WP': 2,
 '150': 6,
 'ppm': 3,
 '18+': 12,
 'New': 12,
 'TEXTBUDDY': 2,
 'Chat': 7,
 '2': 124,
 'horny': 4,
 'guys': 2,
 'area': 5,
 '4': 69,
 'just': 35,
 '25p': 4,
 'Free': 25,
 'receive': 20,
 'Search': 2,
 'postcode': 3,
 'at': 22,
 'gaytextbuddy.com.': 2,
 'TXT': 5,
 'ONE': 3,
 'name': 6,
 '89693': 1,
 'Dear': 13,
 'Voucher': 4,
 'Holder,': 4,
 'To': 56,
 'claim': 57,
 'weeks': 7,
 'of

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [7]:
vocabulary = set()
for text in train_df['text']:
    for word in text.split():
        vocabulary.add(word)
V = len(vocabulary)
V

5977

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [8]:
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [9]:
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
  for class_ in class_word_freq.keys():  
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [10]:
import numpy as np

y_hat_train = X_train.map(lambda x: classify_doc(x, class_word_freq, p_classes, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

False    0.754464
True     0.245536
dtype: float64

## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!